In [1]:
import numpy as np

N, D_in, H, D_out = 64, 1000, 100, 10

x = np.random.randn(N, D_in)
y = np.random.randn(N, D_out)

w1 = np.random.randn(D_in, H)
w2 = np.random.randn(H, D_out)

learning_rate = 1e-6

for t in range(500):
    h = x.dot(w1)
    h_relu = np.maximum(h, 0)
    y_pred = h_relu.dot(w2)

    loss = np.square(y_pred - y).sum()
    if t % 100 == 99:
        print(t, loss)

    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.T.dot(grad_y_pred)
    grad_h_relu = grad_y_pred.dot(w2.T)
    grad_h = grad_h_relu.copy()
    grad_h[h < 0] = 0
    grad_w1 = x.T.dot(grad_h)

    w1 -= grad_w1 * learning_rate
    w2 -= grad_w2 * learning_rate

99 966.8298746717546
199 7.857010470677409
299 0.10263471058515694
399 0.0016685618685175704
499 3.095056065706153e-05


In [2]:
# 日本語訳追記：
# 学習前の重みを定義（前の学習で使用された重みの初期値とは異なります）
w1_unlearned = np.random.randn(D_in, H)
w2_unlearned = np.random.randn(H, D_out)

# 学習前の出力を算出
h = x.dot(w1_unlearned)
h_relu = np.maximum(h, 0)
y_pred = h_relu.dot(w2_unlearned)
print(f"学習前出力：{np.round(y_pred[0], decimals=2)}")

# 学習によって得られた重みで出力を算出
h = x.dot(w1)
h_relu = np.maximum(h, 0)
y_pred = h_relu.dot(w2)
print(f"学習後出力：{np.round(y_pred[0], decimals=2)}")

# 目的の出力yとの比較
print(f"目的の出力：{np.round(y[0], decimals=2)}")

学習前出力：[  30.91 -110.4   -44.61   27.37  -31.97  -99.02  137.56  -80.26  244.61
 -191.8 ]
学習後出力：[-1.34  0.47  0.33  0.1   0.97 -0.05 -0.38  0.22 -1.32  0.17]
目的の出力：[-1.34  0.47  0.33  0.1   0.96 -0.05 -0.38  0.22 -1.32  0.17]


In [3]:
import torch

dtype = torch.float
device = torch.device("cpu")

N, D_in, H, D_out = 64, 1000, 100, 10

x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

w1 = torch.randn(D_in, H, device=device, dtype=dtype)
w2 = torch.randn(H, D_out, device=device, dtype=dtype)

learning_rate = 1e-6

for t in range(500):
    h = x.mm(w1)
    h_relu = h.clamp(min=0)
    y_pred = h_relu.mm(w2)

    loss = (y_pred - y).pow(2).sum().item()
    if t % 100 == 99:
        print(t, loss)
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.t().mm(grad_y_pred)
    grad_h_relu = grad_y_pred.mm(w2.t())
    grad_h = grad_h_relu.clone()
    grad_h[h < 0] = 0
    grad_w1 = x.t().mm(grad_h)

    w1 -= grad_w1 * learning_rate
    w2 -= grad_w2 * learning_rate

99 535.2940063476562
199 3.3119382858276367
299 0.03623326122760773
399 0.0007735778926871717
499 8.576780237490311e-05


In [4]:
import torch

dtype = torch.float
# device = torch.device("cpu")
device = torch.device("cuda")
# GPUで実行する場合はコメントアウトを解除してください。
torch.backends.cuda.matmul.allow_tf32 = False
# GPUで実行する場合はコメントアウトを解除してください。

# 上の行でTensorFloat32を無効にしています。
# TensorFloat32を用いると精度を犠牲にして、ネットワークの高速化を図ることができます。
# TensorFloat32は多くのモデルにおいて有効ですが、このチュートリアルで取り扱う簡易的な
# モデルでは、精度が下がることが原因で損失が収束しないという問題が発生します。
# 詳細については、以下を参照してください：
# https://pytorch.org/docs/stable/notes/cuda.html#tensorfloat-32-tf32-on-ampere-devices

# N：バッチサイズ         D_in：入力層の次元数
# H：隠れ層の次元数       D_out： 出力層の次元数
N, D_in, H, D_out = 64, 1000, 100, 10

# 乱数により入力データと目標となる出力データを保持するTensorを生成
# 逆伝播の際にこのTensorに対する勾配を計算する必要がない場合は、
# requires_grad=Falseを指定します。
# 日本語訳注：デフォルトではrequires_gradはFalseが設定されています。
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# 乱数による重みを表すTensorの定義
# 逆伝播の際、このTensorに対する勾配を計算する場合は、requires_grad=Trueを指定します
w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for t in range(500):
    # 順伝播： Tensorの演算を利用して予測結果yの算出
    # ここはTensorsを使用した順伝播の計算と全く同じ操作ですが、逆伝播の経路を手動で
    # 定義していないので、順伝播の途中の値を保持しておく必要はありません。
    y_pred = x.mm(w1).clamp(min=0).mm(w2)

    # Tensorを用いた損失の計算と表示
    # 損失は形状[1,]のTensorになります。
    # loss.item() は損失を表すTensorの持つ値をスカラー値で返します
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())

    # autograd を使用して逆伝播の計算をします。
    # backward()によりrequires_gradにTrueが設定されているすべてのTensorに対して、
    # 損失の勾配を計算を行います。これにより、w1.gradとw2.gradはそれぞれw1とw2に
    # 対する損失の勾配を保持するTensorになります。
    loss.backward()

    # 確率的勾配降下法を使って手動で重みを更新します。
    # 重みには requires_gradにTrue が設定されていますが、ここではautogradによる
    # 追跡を避ける必要があるので、torch.no_grad()のブロック内で実行します。
    # 別の方法としては、weight.dataとweight.grad.dataを利用する方法があります。
    # tensor.dataはtensorと保存領域を共有するTensorを返しますが、履歴は追跡されません。
    # torch.optim.SGDを利用して追跡を回避することも可能です。
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad

        # 重みの更新後、手動で勾配を0に初期化
        w1.grad.zero_()
        w2.grad.zero_()

99 1265.0361328125
199 17.024276733398438
299 0.3788815140724182
399 0.010919585824012756
499 0.0006168097606860101


In [5]:
import torch


class MyReLU(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        ctx.save_for_backward(input)
        return input.clamp(min=0)

    @staticmethod
    def backward(ctx, grad_output):
        (input,) = ctx.saved_tensors
        grad_input = grad_output.clone()
        grad_input[input < 0] = 0
        return grad_input


dtype = torch.float
device = torch.device("cuda")
torch.backends.cuda.matmul.allow_tf32 = False

N, D_in, H, D_out = 64, 1000, 100, 10

x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for t in range(500):
    relu = MyReLU.apply

    y_pred = relu(x.mm(w1)).mm(w2)

    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())

    loss.backward()

    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad

        w1.grad.zero_()
        w2.grad.zero_()

99 536.7732543945312
199 1.8566772937774658
299 0.01288494374603033
399 0.00028703652787953615
499 4.777481808559969e-05


In [6]:
import torch
import torch.nn as nn

N, D_in, H, D_out = 64, 1000, 100, 10

x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

model = nn.Sequential(
    nn.Linear(D_in, H),
    nn.ReLU(),
    nn.Linear(H, D_out),
)

loss_fn = nn.MSELoss(reduction="sum")

learning_rate = 1e-4

for t in range(500):
    y_pred = model(x)

    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    model.zero_grad()

    loss.backward()

    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad

99 2.3003973960876465
199 0.031019611284136772
299 0.0007913155714049935
399 2.8571841539815068e-05
499 1.3748210676567396e-06


In [8]:
import torch
import torch.nn as nn


class TwoLayerNet(nn.Module):
    def __init__(self, D_in, H, D_out):

        super(TwoLayerNet, self).__init__()

        self.linear1 = nn.Linear(D_in, H)
        self.linear2 = nn.Linear(H, D_out)

    def forward(self, x):
        h_relu = self.linear1(x).clamp(min=0)
        y_pred = self.linear2(h_relu)
        return y_pred


N, D_in, H, D_out = 64, 1000, 100, 10

x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

model = TwoLayerNet(D_in, H, D_out)

criterion = nn.MSELoss(reduction="sum")
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)
for t in range(500):
    y_pred = model(x)

    loss = criterion(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

99 2.93630051612854
199 0.03962613642215729
299 0.0011231000535190105
399 4.731762965093367e-05
499 2.4099006168398773e-06


In [14]:
import random

import torch


class DynamicNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        super(DynamicNet, self).__init__()

        self.input_linear = torch.nn.Linear(D_in, H)
        self.middle_linear = torch.nn.Linear(H, H)
        self.last_linear = torch.nn.Linear(H, D_out)

    def forward(self, x):
        h_relu = self.input_linear(x).clamp(min=0)
        for _ in range(random.randint(0, 4)):
            h_relu = self.middle_linear(h_relu).clamp(min=0)
        y_pred = self.last_linear(h_relu)
        return y_pred


N, D_in, H, D_out = 64, 1000, 100, 10

x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

model = DynamicNet(D_in, H, D_out)
criterion = torch.nn.MSELoss(reduction="sum")
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)
for t in range(500):
    y_pred = model(x)

    loss = criterion(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

99 93.0485610961914
199 11.665759086608887
299 4.807529926300049
399 2.2371082305908203
499 0.16170084476470947
